In [41]:
import pandas as pd
import numpy as np
import os


In [49]:

import xml.etree.ElementTree as ET
my_mapping = {'SITTOR':0, 'PINARB':1, 'ROUGOR':2, 'MESCHA':3, 'MESNON':4, 'ACCMOU':5, 'MESBLE':6}
def parse_doc(xml_file:str):
    final_list = []
    root =  ET.parse(xml_file).getroot()
    l = root.findall("image")
    for element in l:        
        image_dict = {}
        pathh = element.attrib['name']
        k = pathh.split('/')
        k[0] = '/Users/yassinedehbi/ProjetLong/OrnithoMateData/raw_data'
        separator = '/'
        pathh = separator.join(k)
        image_dict['path'] = pathh
        image_dict['width'] = element.attrib['width']
        image_dict['height'] = element.attrib['height']
        image_dict['elements'] = []
        boxes = element.findall("box")
        box_dict = {}
        for box in boxes:
            for attribue in box.findall('.//attribute'):
                if attribue.attrib['name']=="species":
                    if attribue.text == 'noBird' or attribue.text == 'unknown':
                        pass
                    else :

                        box_dict["label"] = my_mapping[attribue.text]
                        box_dict["xtl"] = box.attrib["xtl"]
                        box_dict["ytl"] = box.attrib["ytl"]
                        box_dict["xbr"] = box.attrib["xbr"]
                        box_dict["ybr"] = box.attrib["ybr"]
            
                        image_dict['elements'].append(box_dict)
                else:
                    pass
        final_list.append(image_dict)
    return final_list

In [50]:
l = []
for xml_file in os.listdir('/Users/yassinedehbi/ProjetLong/OrnithoMateData/cvat_annotations_wip'):
    ll = parse_doc('/Users/yassinedehbi/ProjetLong/OrnithoMateData/cvat_annotations_wip/'+xml_file)
    l+=ll

In [51]:
l[15]['elements']

[{'label': 5,
  'xtl': '3.86',
  'ytl': '408.93',
  'xbr': '822.06',
  'ybr': '995.54'}]

In [52]:
labels = []
for khrya in l :
    liste_elements = khrya['elements']
    if len(liste_elements) != 0:
        for boxx in liste_elements:
            #labels.add(box['label'])
            labels.append(boxx['label'])




In [53]:
labels = list(set(labels))

In [54]:
labels

[0, 1, 2, 3, 4, 5, 6]

In [56]:
l[15]

{'path': '/Users/yassinedehbi/ProjetLong/OrnithoMateData/raw_data/task_05-01-2021/2021-01-05-17-41-14.jpg',
 'width': '1920',
 'height': '1088',
 'elements': [{'label': 5,
   'xtl': '3.86',
   'ytl': '408.93',
   'xbr': '822.06',
   'ybr': '995.54'}]}

In [62]:
train_file = open('/Users/yassinedehbi/ProjetLong/train.txt', 'w')
for img in l:
    line = img['path'] + ' '
    if len(img['elements']) !=0:
        for element in img['elements']:
            line += element['xtl'] + ',' + element['ytl'] + ',' + element['xbr'] + ',' +  element['ybr'] + ','+ str(element['label']) + ' '
        train_file.write(line+'\n')

In [ ]:
with open('/Users/yassinedehbi/ProjetLong/train.txt') as t_f:
        t_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(t_lines)
np.random.seed(None)
train_test_pourcentage = len(t_lines)